In [181]:
import numpy as np
import pandas as pd
import networkx as nx
import os

In [198]:
#Load the dataset
XY_file_name = "200p_1000samples_XY.txt";
data_folder_path = "/ihome/hpark/zhf16/causalDeepVASE/data";
dataset = pd.read_csv(data_folder_path+os.path.sep+XY_file_name,sep="\t");
print(dataset.shape)

(1000, 201)


In [199]:
#Load associations
MGM_association_file = "200p_1000samples_XY_MGM_associations.csv";
MGM_associations = pd.read_csv(data_folder_path+os.path.sep+MGM_association_file,index_col=0);
print(MGM_associations.shape);
DNN_association_file = "DNN_selected_associations.csv";
DNN_associations = pd.read_csv(data_folder_path+os.path.sep+DNN_association_file,index_col=0);
print(DNN_associations.shape);
DNN_associations.head(1)

(206, 2)
(78, 2)


,Feature1,Feature2
0,F1,Y


In [200]:
feature_list = ["Y"];
prior_list =  ["F1","F2","F3","F4","F5","F6","F7","F8","F9","F10"];

# for index,row in MGM_associations.iterrows():
#     f1 = row["Feature1"];
#     f2 = row["Feature2"];
#     if (f1=="Y" and f2 in prior_list) or (f2=="Y" and f1 in prior_list):
#         if f1 not in feature_list:
#             feature_list.append(f1);
#         if f2 not in feature_list:
#             feature_list.append(f2);
# print(len(feature_list));

for index,row in DNN_associations.iterrows():
    f1 = row["Feature1"];
    f2 = row["Feature2"];
    if (f1=="Y" and f2 in prior_list) or (f2=="Y" and f1 in prior_list):
        if f1 not in feature_list:
            feature_list.append(f1);
        if f2 not in feature_list:
            feature_list.append(f2);
print(feature_list);

#Only keep features in the selected associations
dataset = dataset[feature_list];
print(dataset.shape);

['Y', 'F1', 'F2', 'F3', 'F4', 'F5', 'F7', 'F9', 'F10']
(1000, 9)


In [201]:
#Map feature names to their numeric representations
fea_name_id_map = {}
fea_name_id_map_rev = {}
for index, name in enumerate(dataset.columns):
    print("Name: {} ID: {}".format(name,index));
    fea_name_id_map[name] = index;
    fea_name_id_map_rev[index] = name;
data_df = dataset.rename(columns=fea_name_id_map);
data_df.head(1)

Name: Y ID: 0
Name: F1 ID: 1
Name: F2 ID: 2
Name: F3 ID: 3
Name: F4 ID: 4
Name: F5 ID: 5
Name: F7 ID: 6
Name: F9 ID: 7
Name: F10 ID: 8


,0,1,2,3,4,5,6,7,8
0,25.982736,-1.204509,0.906381,-0.014023,-2.200692,4.61171,2.963525,2.710397,-4.477634


In [202]:
#Build a skelaton graph using MGM and DNN associations
def build_skeleton_graph():
    
    skeleton_g = nx.Graph();
    
#     #Load the undirect graph returned from MGM
#     for index,row in MGM_associations.iterrows():
#         f1 = row["Feature1"];
#         f2 = row["Feature2"];
#         if (f1=="Y" and f2 in prior_list) or (f2=="Y" and f1 in prior_list):
#             node1 = fea_name_id_map[f1];
#             node2 = fea_name_id_map[f2];
#             skeleton_g.add_edge(node1,node2);
            
    #Load the undirect graph returned from DNN
    for index,row in DNN_associations.iterrows():
        f1 = row["Feature1"];
        f2 = row["Feature2"];
        if (f1=="Y" and f2 in prior_list) or (f2=="Y" and f1 in prior_list):
            node1 = fea_name_id_map[f1];
            node2 = fea_name_id_map[f2];
            skeleton_g.add_edge(node1,node2);
            
    return skeleton_g;

In [203]:
from causal.PC import PC

pc = PC(data_df);

skeleton_graph = build_skeleton_graph();
(g, sep_set) = pc.estimate_skeleton(data_df=data_df, g=skeleton_graph, alpha=0.15);#,method="stable"
g = pc.estimate_cpdag(skel_graph=g, sep_set=sep_set)

In [204]:
fea_name_id_map2 = {y:x for x,y in fea_name_id_map.items()};
G = nx.relabel_nodes(g, fea_name_id_map2);

edge_list = [];
for edge in G.edges():
    edge_list.append({"cause":edge[0],"effect":edge[1]});
print(len(edge_list));

8


In [205]:
pd.DataFrame(data=edge_list)#.to_csv("PC_result_on_DeepPINK_MGM_p"+str(p)+"_v4.csv")

,cause,effect
0,F1,Y
1,Y,F1
2,Y,F2
3,Y,F3
4,Y,F4
5,F2,Y
6,F3,Y
7,F4,Y
